In [0]:
import yaml
from pyspark.sql import SparkSession
import os
from pyspark.sql.types import IntegerType, FloatType, DateType
from pyspark.sql.functions import col
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# --- 1. Environment Configuration ---
try:
    # Get environment variable from Databricks widget or set default
    ENV = dbutils.widgets.get("env_name")
except Exception:
    ENV = 'TEST'
    
# --- 2. Load Configuration ---
try:
    # Load configuration from the YAML file
    with open('../../config/config.yaml', 'r') as file:
        full_config = yaml.safe_load(file)
except FileNotFoundError:
    print("ERROR: 'config.yaml' file not found! Check the path.")
    raise

CFG = full_config.get(ENV)
if not CFG:
    raise ValueError(f"Configuration not found for environment: {ENV} in YAML file.")

catalog_name = CFG['catalog_name']
schema_name = CFG['schema_name']
volume_name = CFG['volume_name']

# Define the base path for the Bronze table
base_output_directory = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/yfinance_bronze_data"
print(f"Path to the Bronze Delta Table: {base_output_directory}")

In [0]:
# --- BRONZE LAYER: Read & Type Optimization ---

# Read data from the Bronze Delta table
try:
    # Use "delta" format to read the Delta Lake folder
    df_bronze = (
        spark.read.format("delta") 
        .load(base_output_directory) # Load from the Delta folder path
        .withColumn("Date", col("Date").cast(DateType()))
    )
    
    print(f"Successfully loaded data from Bronze. Row count: {df_bronze.count()}")
    
except Exception as e:
    print(f"ERROR: An issue occurred while loading data: {e}")
    raise e

In [0]:
# Apply data type optimization
df_optimized = (
    df_bronze
    .withColumn("Date", col("Date").cast(DateType())) # Date: remains 'date'
    
    # Change from 'double' to 'float' (for smaller memory footprint)
    .withColumn("Open", col("Open").cast(FloatType()))
    .withColumn("High", col("High").cast(FloatType()))
    .withColumn("Low", col("Low").cast(FloatType()))
    .withColumn("Close", col("Close").cast(FloatType()))
    
    # Change from 'long' to 'integer'
    .withColumn("Volume", col("Volume").cast(IntegerType())) 
    # 'Ticket' and 'company_name' remain 'string'
)

# Replace the original DataFrame with the optimized one
df_bronze = df_optimized

print(f"Schema optimized.")
df_bronze.printSchema()

In [0]:
# --- BRONZE LAYER: Data Quality Checks (Nulls and Duplicates) ---

# 1. Minimal Validation for Nulls in Critical Columns
critical_null_count = df_bronze.filter(
    F.col("Date").isNull() | 
    F.col("Ticket").isNull() | 
    F.col("Close").isNull()
).count()

if critical_null_count > 0:
    # Raise an error to stop the ETL pipeline and require intervention
    raise ValueError(f"QA ERROR: Found {critical_null_count} rows with NULLs in critical columns (Date, Ticket, Close). Pipeline stopped.")

In [0]:
# 2. Duplicate Check based on Date & Ticket
deduplication_key = ["Date", "Ticket"]
total_rows = df_bronze.count()
df_unique = df_bronze.dropDuplicates(subset=deduplication_key)
unique_rows = df_unique.count()
duplicate_count = total_rows - unique_rows
duplicate_percentage = (duplicate_count / total_rows) * 100

print(f"\n--- Duplicate Report ---")
print(f"Total rows: {total_rows}")
print(f"Unique rows: {unique_rows}")
print(f"Duplicates found: {duplicate_count}")
print(f"Duplicate percentage: {duplicate_percentage:.2f}%")

# IMPLEMENTATION OF PIPELINE HALT ON DUPLICATES
if total_rows != unique_rows:
    # Use ValueError to halt the pipeline and flag the error
    raise ValueError(
        f"QA ERROR: Duplicates found in key (Date, Ticket)! "
        f"Total duplicates: {duplicate_count} rows ({duplicate_percentage:.2f}%). "
        f"ETL pipeline halted for source data inspection."
    )
    
# If no duplicates, proceed with the unique DataFrame
df_bronze = df_unique 
print("DUPLICATE VALIDATION: SUCCESS. Proceeding to Silver layer.")

In [0]:
# --- SILVER LAYER: Feature Engineering & Transformation ---

PRECISION = 4 # Use the same precision constant

# 1. Calculate Daily Returns (Pct Change)

# Define Window: Partition by Ticket, ordered by Date
window_spec = (
    Window.partitionBy("Ticket")
    .orderBy("Date")
)

df_silver = (
    df_bronze
    .withColumn(
        "Previous_Close", 
        # Calculate the close price from the previous trading day
        F.lag(F.col("Close"), 1).over(window_spec)
    )
    .withColumn(
        "Daily_Return_Pct", 
        # Calculate percentage change and explicitly round to PRECISION places
        F.round(
            ((F.col("Close") - F.col("Previous_Close")) / F.col("Previous_Close")) * 100, 
            PRECISION # Use the constant 4
        ).cast(FloatType())
    )
    .drop("Previous_Close") # Remove the temporary column
)

print(f"\nSuccessfully calculated and rounded 'Daily_Return_Pct' to {PRECISION} decimal places.")


In [0]:
# 2. Add Time Dimension Columns
df_silver = (
    df_silver
    .withColumn(
        "Year", 
        F.year(F.col("Date")) # Extracts year (e.g., 2023)
    )
    .withColumn(
        "Quarter", 
        F.quarter(F.col("Date")) # Extracts quarter (1, 2, 3, or 4)
    )
    .withColumn(
        "Month", 
        F.month(F.col("Date")) # Extracts month (1 to 12)
    )
    .withColumn(
        "WeekOfYear", 
        # Extracts week number in the year (1 to 53)
        F.weekofyear(F.col("Date")) 
    )
)

print("Successfully added Year, Quarter, Month, WeekOfYear columns.")

In [0]:
# 3. Calculate Simple Moving Averages (SMA)

SMA_PERIODS = [20, 50, 200]
base_window_spec = (
    Window.partitionBy("Ticket")
    .orderBy("Date")
)

df_silver_sma = df_silver

# Iteratively create SMA columns
for N in SMA_PERIODS:
    # Define Window Specification for period N
    # rowsBetween(-(N-1), 0) means: from N-1 rows back up to the current row (0)
    window_spec_n = base_window_spec.rowsBetween(-(N - 1), 0)

    column_name = f"SMA_{N}"
    
    # Calculate Simple Moving Average (SMA)
    df_silver_sma = df_silver_sma.withColumn(
        column_name,
        # Use avg() function on 'Close' column within the defined window
        F.avg(F.col("Close")).over(window_spec_n).cast(FloatType())
    )
df_silver = df_silver_sma

print(f"Successfully added SMA columns for periods: {SMA_PERIODS}")

In [0]:
# --- SILVER LAYER: Write to Delta Lake ---

# Path for Silver layer
silver_table_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/yfinance_silver_data"

# Ensure the directory exists
try:
    dbutils.fs.mkdirs(silver_table_path)
    print(f"Successfully created directory for Silver: {silver_table_path}")
except Exception as e:
    print(f"Silver directory already exists or error occurred: {e}")

# Recommended write: overwrite + partitioning
(
    df_silver.write
    .format("delta")
    .mode("overwrite")
    .partitionBy("Ticket", "Year") # Partitioning by Ticket and Year for query optimization
    .option("overwriteSchema", "true")
    .save(silver_table_path)
)

print(f"Successfully saved data to Silver layer at: {silver_table_path}")